#### Разные хитрые штучки для ускоренной работы с PostgreSQL

In [ ]:
! pip install psycopg2-binary

Found existing installation: psycopg2 2.9.10
Uninstalling psycopg2-2.9.10:
  Would remove:
    /media/gansior/t128/python_workshops/.py_venv/lib/python3.12/site-packages/psycopg2-2.9.10.dist-info/*
    /media/gansior/t128/python_workshops/.py_venv/lib/python3.12/site-packages/psycopg2/*
Proceed (Y/n)? ERROR: Operation cancelled by user
^C


In [1]:
import psycopg2
from psycopg2 import sql

In [2]:
import config as cf

In [3]:
connection = psycopg2.connect(
    host=cf.host,            # или IP-адрес сервера БД
    port=cf.port,            # порт (по умолчанию 5432)
    database=cf.database,    # имя вашей базы данных
    user=cf.user,            # имя пользователя PostgreSQL
    password=cf.password     # пароль
)
cur = connection.cursor()

In [4]:
sql = """CREATE TABLE proba.adb_stat_dev (
            "source" text NULL,
            event_time timestamp NULL,
            schema_name text NULL,
            table_name text NULL,
            operation_type text NULL,
            hash_diff text NOT NULL
        );"""
cur.execute(sql)
connection.commit()

##### Способ 1 

In [4]:
def write_db(row_data, connection, cur):
    sql = """INSERT INTO proba.adb_stat_dev (source, event_time,
            schema_name, table_name, operation_type,
            hash_diff) VALUES (%s, %s, %s,%s, %s, %s);"""
    cur.execute(sql,row_data)
    connection.commit()
            

In [5]:

with open('dataset/adb_stat_dev.csv', 'r') as r_f:
    next(r_f)  # ← пропустить первую строку (заголовок)
    for line in r_f:
        ss = line.strip()
        if not ss:
            continue
        mas = ss.split(',')
        if len(mas) != 6:
            print("Неверное количество полей:", mas)
            continue
        write_db(mas, connection, cur)

##### Способ 2

In [4]:
# Загружаем напрямую
with open('dataset/adb_stat_dev.csv', 'r') as f:
    cur.copy_expert(
        "COPY proba.adb_stat_dev FROM STDIN WITH (FORMAT CSV, HEADER)",
        f
    )
connection.commit()

##### Способ 3

In [6]:
import csv
from io import StringIO

In [8]:
# Чтение CSV и загрузка через COPY
with open('dataset/adb_stat_dev.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # пропустить заголовок
    
    # Записываем данные во временный буфер
    output = StringIO()
    writer = csv.writer(output)
    for row in reader:
        if len(row) == 6:
            writer.writerow(row)
    output.seek(0)

    # Загружаем в БД
    cur.copy_expert(
        "COPY proba.adb_stat_dev FROM STDIN WITH (FORMAT CSV)",
        output
    )
    connection.commit()

##### Способ 4

In [4]:
from psycopg2.extras import execute_batch

In [5]:
rows = []
with open('dataset/adb_stat_dev.csv', 'r') as r_f:
    next(r_f)  # ← пропустить первую строку (заголовок)
    for line in r_f:
        ss = line.strip().split(',')
        if len(ss) == 6:
                    rows.append(ss)
execute_batch(
    cur,
    """INSERT INTO proba.adb_stat_dev ("source", event_time, schema_name, table_name, operation_type, hash_diff)
       VALUES (%s, %s, %s, %s, %s, %s)""",
    rows,
    page_size=1000
)
connection.commit()

In [6]:
# Контекст на БД

In [4]:
class DB:
    def __init__(self, host='localhost', port=5432, database='', user='', password=''):
        self.host = host
        self.port = port
        self.database = database
        self.user = user
        self.password = password
        
    def __enter__(self):
        self.connection = psycopg2.connect(
            host=self.host,            # или IP-адрес сервера БД
            port = self.port,          # порт (по умолчанию 5432)
            database=self.database,    # имя вашей базы данных
            user=self.user,            # имя пользователя PostgreSQL
            password=self.password     # пароль
        )
        self.cur = self.connection.cursor()
        return self.cur
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.commit()
        self.conn.close()
        


In [ ]:
class WorkDB:
    
    def queryDB(fQuery):
        def workers(*args):
            with DB(DbPath) as db:
                ss =fQuery()
                db.execute(ss)
        return workers  
        
    @classmethod
    @queryDB
    def createrTb():
        sql = """CREATE TABLE gansior.adb_stat_dev (
            "source" text NULL,
            event_time timestamp NULL,
            schema_name text NULL,
            table_name text NULL,
            operation_type text NULL,
            hash_diff text NOT NULL
        );"""
        return sql
    
    @classmethod
    @queryDB
    def InsertDataTb_sourcesData():
        sql = "INSERT INTO sourcesData ("\
            "guidsources,"\
            "path_sours, "\
            "data_start"\
            ")"\
            " VALUES ('1', 'sdfgsdfg', 'sdfgsdfg'),"\
            "('2', 'sdfgsdfg', 'sdfgsdfg'),"\
            "('3', 'sdfgsdfg', 'sdfgsdfg'"\
            ");"
        return sql